# Building and Training our own ResNet34 model

Follow this paper https://arxiv.org/pdf/1908.04913v1.pdf, we decide to use the UTK face dataset and ResNet34 to build our own model for thie project. Below is the steps to build the model.

In [23]:
!pip install imutils

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/pkg.7/python3/3.8.6/install/bin/python3.8 -m pip install --upgrade pip' command.


In [24]:
import tensorflow as tf
import numpy as np

# Definition of the identity block of the ResNet34
def identity_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x


# Definition of Convolutional Block of the ResNet34
def convolutional_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same', strides = (2,2))(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Processing Residue with conv(1,1)
    x_skip = tf.keras.layers.Conv2D(filter, (1,1), strides = (2,2))(x_skip)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x


# Definition of the ResNet34
def ResNet34(shape = (200, 200, 3), classes = 4):
    # Step 1 (Setup Input Layer)
    x_input = tf.keras.layers.Input(shape)
    x = tf.keras.layers.ZeroPadding2D((3, 3))(x_input)
    # Step 2 (Initial Conv layer along with maxPool)
    x = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    # Define size of sub-blocks and initial filter size
    block_layers = [3, 4, 6, 3]
    filter_size = 64
    # Step 3 Add the Resnet Blocks
    for i in range(4):
        if i == 0:
            # For sub-block 1 Residual/Convolutional block not needed
            for j in range(block_layers[i]):
                x = identity_block(x, filter_size)
        else:
            # One Residual/Convolutional Block followed by Identity blocks
            # The filter size will go on increasing by a factor of 2
            filter_size = filter_size*2
            x = convolutional_block(x, filter_size)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filter_size)
    # Step 4 End Dense Network
    x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512, activation = 'relu')(x)
    x = tf.keras.layers.Dense(classes, activation = 'softmax')(x)
    model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "ResNet34")
    return model

## Preprocessing
    We use the dataset UTKFace to train our database, the data set contains totally 23708 images, we randomly select 2708 of them as validation set, 500 of them as test set. The name of each image already contains the label info of the image: the format of the label is [age]_[gender]_[race]_[date&time].jpg.
    In the preprocessing, we do:
    1. Read the image and turn them into rgb.
    2. Shuffle the train set.
    3. Normalize the train set and test set, by dividing each pixil value by 255, where 255 is the max value of each pixel.
    

In [72]:
import random
import cv2

from imutils import paths


image_original_folder = "../resource/UTKFace_filtered/"
image_paths = list(paths.list_images(image_original_folder))

test_set_size = 100 # There are total 23708 images in UTKFace, so this should not exceed 23708
non_training_data = random.sample(range(0, 10446), test_set_size)
test_indexes = set(non_training_data)

train_set = []
train_labels = []
test_set = []
test_labels = []
for (i, imagePath) in enumerate(image_paths):
    if i in test_indexes:
        added_set = test_set
        added_labels = test_labels
    else:
        added_set = train_set
        added_labels = train_labels
    # the format of the label is [age]_[gender]_[race]_[date&time].jpg,
    # here only age and race are needed
    labels = imagePath.split("/")[-1].split(".")[0].split("_")
    if len(labels) != 4:
        # few images in the UTKFace have broken labels, ignore them
        continue
    labels = [int(labels[0])]
    label = 0 if labels[0] < 21 else 1
    
    image = cv2.imread(imagePath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    added_set.append(gray)
    added_labels.append(label)

print(len(train_set))
print(len(train_labels))
print(len(test_set))
print(len(test_labels))




10346
10346
100
100


In [73]:
# shuffle the train data
train_XY = list(zip(train_set, train_labels))
random.shuffle(train_XY)

train_set, train_labels = zip(*train_XY)
train_set = list(train_set)
train_labels = list(train_labels)


In [74]:
# normalize the data image
train_set = np.array(train_set)
train_labels = np.array(train_labels)
test_set = np.array(test_set)
test_labels = np.array(test_labels)
normed_train_set = train_set/255
normed_test_set = test_set/255

## Train the model
    Besides training the model, we also use the test set to see the performance of the model

In [76]:
# TODO: should we nnormalize the image vectors?

num_epochs = 32
learning_rate = 0.001
batch_size = 32
model = ResNet34(shape = (200, 200, 1) ,classes=2)
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(
                    initial_learning_rate=learning_rate, decay_steps=num_epochs*100, decay_rate=0.9)), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])
model.fit(normed_train_set, train_labels, epochs=num_epochs, batch_size=batch_size)

Epoch 1/32
324/324 [==============================] - 32s 99ms/step - loss: 0.6208 - accuracy: 0.6640
Epoch 2/32
324/324 [==============================] - 32s 100ms/step - loss: 0.5346 - accuracy: 0.7678
Epoch 3/32
324/324 [==============================] - 32s 99ms/step - loss: 0.5067 - accuracy: 0.7969
Epoch 4/32
324/324 [==============================] - 32s 99ms/step - loss: 0.4865 - accuracy: 0.8211
Epoch 5/32
324/324 [==============================] - 32s 99ms/step - loss: 0.4701 - accuracy: 0.8383
Epoch 6/32
324/324 [==============================] - 32s 99ms/step - loss: 0.4606 - accuracy: 0.8456
Epoch 7/32
324/324 [==============================] - 32s 99ms/step - loss: 0.4458 - accuracy: 0.8657
Epoch 8/32
324/324 [==============================] - 32s 99ms/step - loss: 0.4352 - accuracy: 0.8754
Epoch 9/32
324/324 [==============================] - 32s 99ms/step - loss: 0.4269 - accuracy: 0.8859
Epoch 10/32
324/324 [==============================] - 32s 99ms/step - loss: 0.41

In [77]:
model.save("2021_12_4_15_46")
model = tf.keras.models.load_model("2021_12_4_15_46")

INFO:tensorflow:Assets written to: 2021_12_4_15_46/assets


In [78]:
from sklearn import metrics
from sklearn.metrics import classification_report

prediction_prob = model.predict(normed_test_set)

In [79]:
results = tf.argmax(prediction_prob, axis=1)
metrics.accuracy_score(test_labels, results)
print(classification_report(y_true=test_labels,y_pred=results))

              precision    recall  f1-score   support

           0       0.97      0.60      0.74        48
           1       0.73      0.98      0.84        52

    accuracy                           0.80       100
   macro avg       0.85      0.79      0.79       100
weighted avg       0.84      0.80      0.79       100



## Predict on the twitter user profile
    Below we will run the age prediction on the first batch of the twitter user profile image.
    The preprocess steps are same as what have been down to the training set.

In [80]:
profile0_folder = "../resource/cropped_profile/"
image_paths = list(paths.list_images(profile0_folder))

test_set0 = []
test_labels0 = []
for (i, imagePath) in enumerate(image_paths):
    # the format of the label is [age]_[race]_[id].jpeg,
    # here only age and race are needed
    labels = imagePath.split("/")[-1].split(".")[0].split("_")
    if len(labels) != 3:
        # few images in the UTKFace have broken labels, ignore them
        continue
    labels = [int(labels[0])]
    label = 0 if labels[0] < 21 else 1
    
    image = cv2.imread(imagePath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    test_set0.append(gray)
    test_labels0.append(label)

test_set0 = np.array(test_set0)
test_labels0 = np.array(test_labels0)
normed_test_set0 = test_set0/255

prediction_prob = model.predict(normed_test_set0)
results = tf.argmax(prediction_prob, axis=1)
print(metrics.accuracy_score(test_labels0, results))
print(classification_report(y_true=test_labels0,y_pred=results))



0.92312040700961
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       1.00      0.93      0.96      1762

    accuracy                           0.92      1769
   macro avg       0.50      0.46      0.48      1769
weighted avg       0.99      0.92      0.96      1769



## Predict on the second user profile set

In [81]:
profile1_folder = "../resource/cropped_profile1/"
image_paths = list(paths.list_images(profile1_folder))

test_set1 = []
test_labels1 = []
for (i, imagePath) in enumerate(image_paths):
    # the format of the label is [age]_[id].jpeg,
    # here only age and race are needed
    labels = imagePath.split("/")[-1].split(".")[0].split("_")
    if len(labels) != 2:
        # few images in the UTKFace have broken labels, ignore them
        continue
    labels = [int(labels[0])]
    label = 0 if labels[0] < 21 else 1
    
    image = cv2.imread(imagePath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    test_set1.append(gray)
    test_labels1.append(label)

test_set1 = np.array(test_set1)
test_labels1 = np.array(test_labels1)
normed_test_set1 = test_set1/255

prediction_prob = model.predict(normed_test_set1)
results = tf.argmax(prediction_prob, axis=1)
print(metrics.accuracy_score(test_labels1, results))
print(classification_report(y_true=test_labels1,y_pred=results))

0.586608442503639
              precision    recall  f1-score   support

           0       0.38      0.09      0.15       269
           1       0.61      0.90      0.73       418

    accuracy                           0.59       687
   macro avg       0.50      0.50      0.44       687
weighted avg       0.52      0.59      0.50       687

